In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
from shapely import wkt
from shapely.geometry import LineString
import geopandas

ImportError: cannot import name 'CRS' from 'pyproj' (C:\Users\Antonis\anaconda3\lib\site-packages\pyproj\__init__.py)

In [5]:
walkability_df = pd.read_csv('data/WALKABILITY.csv', sep=';')
walkability_df

,OBJECTNUMMER,Straatvaknummer,Loopruimte,Walkability_algemeen,Drukte_algemeen,Walkability_recreatief,Drukte_recreatief,Walkability_toeristisch,Drukte_toeristisch,Walkability_werkgerelateerd,Drukte_werkgerelateerd,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Unnamed: 15
0,1,1,ruim,goed,druk,matig,zeer druk,uitstekend,zeer rustig,uitstekend,zeer rustig,"LINESTRING(4.88493 52.342693,4.884867 52.342696)","LINESTRING(52.342693 4.88493,52.342696 4.884867)",4.884899,52.342695,NaN
1,2,2,zeer ruim,uitstekend,zeer rustig,uitstekend,zeer rustig,uitstekend,zeer rustig,uitstekend,zeer rustig,"LINESTRING(4.93276 52.354894,4.932682 52.354946)","LINESTRING(52.354894 4.93276,52.354946 4.932682)",4.932721,52.354920,NaN
2,3,3,ruim,goed,rustig,goed,rustig,matig,druk,matig,druk,"LINESTRING(4.889813 52.375347,4.889822 52.37535)","LINESTRING(52.375347 4.889813,52.37535 4.889822)",4.889818,52.375349,NaN
3,4,4,ruim,matig,zeer druk,matig,zeer druk,matig,zeer druk,matig,zeer druk,"LINESTRING(4.89343 52.369933,4.893282 52.369381)","LINESTRING(52.369933 4.89343,52.369381 4.893282)",4.893356,52.369657,NaN
4,5,5,krap,goed,zeer rustig,goed,zeer rustig,matig,rustig,matig,rustig,"LINESTRING(4.883258 52.375285,4.88322 52.37530...","LINESTRING(52.375285 4.883258,52.375303 4.8832...",4.883119,52.375293,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9609,9613,9613,krap,goed,zeer rustig,goed,zeer rustig,matig,rustig,matig,rustig,"LINESTRING(4.865319 52.370139,4.864897 52.370283)","LINESTRING(52.370139 4.865319,52.370283 4.864897)",4.865108,52.370211,NaN
9610,9614,9614,ruim,goed,rustig,matig,druk,goed,rustig,goed,rustig,"LINESTRING(4.852033 52.370128,4.852056 52.3700...","LINESTRING(52.370128 4.852033,52.370098 4.8520...",4.852056,52.370120,NaN
9611,9615,9615,krap,matig,rustig,matig,rustig,matig,druk,matig,rustig,"LINESTRING(4.889079 52.375654,4.889084 52.375646)","LINESTRING(52.375654 4.889079,52.375646 4.889084)",4.889081,52.375650,NaN
9612,9616,9616,ruim,uitstekend,zeer rustig,uitstekend,zeer rustig,goed,rustig,goed,rustig,"LINESTRING(4.89696 52.36207,4.897011 52.362065...","LINESTRING(52.36207 4.89696,52.362065 4.897011...",4.897004,52.362093,NaN


In [17]:
walkability_df['WKT_LNG_LAT'] = geopandas.GeoSeries.from_wkt(walkability_df['WKT_LNG_LAT'])
walkability_df['WKT_LAT_LNG'] = geopandas.GeoSeries.from_wkt(walkability_df['WKT_LAT_LNG'])
walkability_df
# walkability_df['Drukte_algemeen'].astype(LineString)

NameError: name 'geopandas' is not defined